In [1]:

import numpy as np
import pandas as pd
from pandas import DataFrame
import gc
import timeit
from Bio import SeqIO
from joblib import dump, load
import multiprocessing as mp
import time
%matplotlib inline
import pickle
from tqdm.auto import tqdm
import ipywidgets
tqdm.pandas()
from ipywidgets import IntProgress
from Bio import SeqIO
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectPercentile, chi2, SelectFromModel

# Step 3 Load the space separated kmers of kmers into dataframe, to be fed to Count Vectorizer
### once done, kill the kernel to free memory.
Now your features are extracted and you can later load the dataframe from disk for training model 
# !Attention, for k>=8, a ~30GiB of Memory is needed
MemoryError: Unable to allocate 29.3 GiB for an array with shape (60000, 65536) and data type int64
To mitigate this, we can write the compact matrix X_cv directly to disk (6.5GB for k=8) like in section 2 below, and load it later transforming it into a DataFrame using slicing and use  SGDClassifier to partially train a model with each batch (slice) that fits in memory (10 batches for example).

In [2]:
###############################################################################
#        Conversion step 3: Use Word Vectorizer to count kmer occurences      #
###############################################################################
#precise size of k from previous steps
k=4

# enable automatic garbage collector to help free unused memory
gc.enable()
# gc.disable()
start = time.time()
###############################################################################
#   load file containing space separated kmers, the result of the last step   #
###############################################################################

with open(r"D:\DataSet\TfIdf\spaced words-"+str(k), 'rb') as filehandle:
     textword = pickle.load(filehandle)
print("number of samples loaded :", len(textword))

cv = CountVectorizer() # Default ngram_range (1, 1)

###############################################################################
#in case of k=1, one character word, use this instead
###############################################################################
# cv = CountVectorizer(analyzer='char') # 

X_cv = cv.fit_transform(textword)

# Get the header, the name of columns
cols=cv.get_feature_names()

# Transform into Pandas Data Frame
X = pd.DataFrame(X_cv.toarray(), columns=cols)

#  View the converted matrix
X.head(10)

###############################################################################
#Write resulting Data Frame on disk. Feather is very fast format for loading  #
###############################################################################
X.reset_index(drop = True).to_feather(r"D:\DataSet\TfIdf\dfk"+str(k)+".feather")

# total time taken
end = time.time()
print(f"Runtime of kmer counting of size: "+ str(k)+ f" using Word Vectorizer is {end - start} secs")

number of samples loaded : 60000
Runtime of kmer counting of size: 4 using Word Vectorizer is 589.4460277557373 secs


#### K=1 : Runtime of kmer counting is 324 secs
#### K=2 : Runtime of kmer counting is 368 secs
#### K=3 : Runtime of kmer counting is 491  secs
#### K=4 : Runtime of kmer counting is 655 secs
  
#### K=5 : Runtime of kmer counting is 827 secs
#### K=6 : Runtime of kmer counting is 1041secs
#### K=7 : Runtime of kmer counting is 1467 secs
